In [1]:
import spacy
import checklist
from checklist.perturb import Perturb
from checklist.test_types import MFT, INV, DIR
from checklist.editor import Editor
from checklist.pred_wrapper import PredictorWrapper
import numpy as np
from pattern.en import sentiment

ModuleNotFoundError: No module named 'checklist'

In [2]:
nlp = spacy.load("en_core_web_sm")
editor = Editor()

64


In [3]:
def predict_proba(inputs):
    p1 = np.array([(sentiment(x)[0] + 1)/2. for x in inputs]).reshape(-1, 1)
    p0 = 1- p1
    return np.hstack((p0, p1))

In [4]:
# wrapped_pp returns a tuple with (predictions, softmax confidences)
wrapped_pp = PredictorWrapper.wrap_softmax(predict_proba)

In [5]:
dataset = ['This was a very nice movie directed by John Smith.',
           'Mary Keen was brilliant.', 
          'I hated everything about this.',
          'This movie was very bad.',
          'I really liked this movie.',
          'just bad.',
          'amazing.',
          ]
pdataset = list(nlp.pipe(dataset))

In [14]:
t = Perturb.perturb(pdataset, Perturb.change_names)
print('\n'.join(t.data[0][:3]))
print('...')
test = INV(**t)
test.run(wrapped_pp)
test.summary()

This was a very nice movie directed by John Smith.
This was a very nice movie directed by Michael Taylor.
This was a very nice movie directed by Christopher Green.
...
Predicting 22 examples
Test cases:      2
Fails (rate):    0 (0.0%)


In [7]:
t = Perturb.perturb(dataset, Perturb.add_typos)
print('\n'.join(t.data[0][:3]))
print('...')
test = INV(**t)
test.run(wrapped_pp)
test.summary()

This was a very nice movie directed by John Smith.
This wsa a very nice movie directed by John Smith.
...
Predicting 14 examples
Test cases:      7
Fails (rate):    1 (14.3%)

Example fails:
0.8 amazing.
0.5 amazin.g

----
